In [ ]:
import numpy as np
import obspy
from obspy import UTCDateTime
from ccfj import CC
from ccfj import GetStationPairs
from concurrent.futures import ThreadPoolExecutor
import os
import time
from geopy.distance import great_circle
import shutil
import pandas as pd

In [ ]:
outdir = 'data_resample_1/'
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [ ]:
nThreads = 50

### Parameters

In [ ]:
#dir_SAC = '/home/harmon/data/data/SJFZ_nodal/2014/'
dir_SAC = '/share/data/California/SJFZ_nodal/2014/'
#stalistname = "sublist/stalist-2007_1" 
stalistname = 'stations_info'

y_start = 2014
y_end = 2015
#d_start = 132
d_start = 132
#d_end = 159
d_end = 133
fmax = 100                    # 降采样频率

### Correlation day by day

In [ ]:
def Checkdata(dirname):
    global day
    global dir_SAC
    filename = os.path.join(dir_SAC,str(day),dirname+'.EHZ.2014.'+str(day)+'.00.00.00')
    if os.path.exists(filename):
        return True
    return False


# Set the rules for reading data
# this will determined by the data files' distribution
def Resample(i):
    global day
    global dir_SAC
    global namelist
    global fmax


    if i%200 == 0 and i != 0:
        print(i)

    dirname = namelist[i]
    filename = os.path.join(dir_SAC,str(day),dirname+'.EHZ.2014.'+str(day)+'.00.00.00')
    st = obspy.read(filename)

    # 设置低通滤波的截止频率
    lowpass_corner_frequency = 0.5 * fmax # 以Nyquist频率的40%设置

    # 对每个通道的数据进行低通滤波
    st_lowpass_filtered = st.copy().filter('lowpass', freq=lowpass_corner_frequency)

    # 对每个通道的数据进行重采样
    st_resampled = st_lowpass_filtered.copy().resample(sampling_rate=fmax)
    
    # 保存到outdir中
    st_resampled.write(os.path.join(outdir,str(day),dirname+'.EHZ.2014.'+str(day)+'.00.00.00'),format='SAC')

In [ ]:
stainfo = pd.read_excel(stalistname+'.xlsx')
nsta = len(stainfo.iloc[:,0])
StationPairs = GetStationPairs(nsta)
nPairs = int(len(StationPairs)/2)
stalist = stainfo['Station'].tolist()

In [ ]:
days = []
years = set()
flag_d = 0
start = time.time()
for y in range(y_start,y_end):
    for d in range(d_start,d_end): 

        if not os.path.exists(outdir+str(d)):
            os.mkdir(outdir+str(d))

        year = str(y)
        day = "%03d"%d
        namelist = []
        for dirname in stalist:
            if Checkdata(dirname):
                namelist.append(dirname)
        nsta = len(namelist)
        # at least two stations needs
        if nsta >1:
            flag_d += 1
            years.add(int(year))
            days.append(d)
            print("year"+year+" day"+day +" nsta "+ str(nsta))
            # using multiThreads to read data
            pool = ThreadPoolExecutor(max_workers = nThreads)
            start0 = time.time()
            for i in range(nsta):
                pool.submit(Resample,i)
            pool.shutdown()